In [ ]:
#final for addresses fetch of particuler token automatically
import requests
import pandas as pd
import json
import os
from IPython.display import display

# Define constants
BASE_URL = 'https://api.geckoterminal.com/api/v2'
HEADERS = {'Accept': 'application/json;version=20230302'}

# Function to safely request data from the API
def safe_request(url, headers, params=None):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as err:
        print(f'HTTP error: {err}')
    except Exception as err:
        print(f'Error: {err}')
    return None

# Function to get pools for a given token
def get_pools_for_token(network, token_address):
    endpoint = f"/networks/{network}/tokens/{token_address}/pools"
    url = BASE_URL + endpoint
    all_pools_data = []
    page = 1
    while True:
        params = {'page': page}
        pools = safe_request(url, HEADERS, params=params)
        if not pools or 'data' not in pools:
            break
        all_pools_data.extend(pools['data'])
        page += 1
    return all_pools_data

# Function to extract relevant information and update JSON file
def process_token(network, token_address, token_name, other_tokens):
    all_pools = get_pools_for_token(network, token_address)

    # Extract relevant information into a dictionary
    data_dict = {
        'base_token_id': [],
        'quote_token_id': [],
        'dex_id': [],
        'name': [],
        'id': [],
        
    }

    for item in all_pools:
        attributes = item['attributes']
        volume_usd = attributes['volume_usd']
        
        data_dict['name'].append(str(attributes['name']))  # Convert to string
        data_dict['base_token_id'].append(item['relationships']['base_token']['data']['id'])
        data_dict['quote_token_id'].append(item['relationships']['quote_token']['data']['id'])
        data_dict['dex_id'].append(item['relationships']['dex']['data']['id'])
        data_dict['id'].append(item['id'])
    

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data_dict)

    # Filter the DataFrame to remove rows with three or more tokens
    df_address_want = df[['name', 'base_token_id', 'quote_token_id']]
    df_address_want = df_address_want[df_address_want['name'].astype(str).str.count('/') < 2]  # Convert to string before using .str accessor

    # Detect WETH token address
    weth_address = network + "_" + token_address

    # Extract rows where WETH is either the base or quote token
    weth_pairs = df_address_want[(df_address_want['base_token_id'] == weth_address) | (df_address_want['quote_token_id'] == weth_address)]

    # Extract other token names and addresses
    for _, row in weth_pairs.iterrows():
        if row['base_token_id'] == weth_address:
            other_token_address = row['quote_token_id']
            other_token_name = row['name'].split(' / ')[1]
        else:
            other_token_address = row['base_token_id']
            other_token_name = row['name'].split(' / ')[0]
        
        if other_token_address in other_tokens:
            if other_tokens[other_token_address] != other_token_name:
                print(f"Conflict detected for address {other_token_address}: {other_tokens[other_token_address]} vs {other_token_name}")
                # Skip adding conflicting token names
                continue
        else:
            other_tokens[other_token_address] = other_token_name
            print("update")

    # Save to JSON file
    with open(file_name, 'w') as f:
        json.dump(other_tokens, f, indent=4)


# Load existing token addresses and names from JSON file if it exists
file_name = 'eth_addresses.json'#bsc_addresses.json
if os.path.exists(file_name):
    with open(file_name, 'r') as f:
        other_tokens = json.load(f)
        
data=other_tokens

# Load existing token addresses and names from JSON file if it exists
file_name = 'eth_addresses.json'#bsc_addresses.json
if os.path.exists(file_name):
    with open(file_name, 'r') as f:
        other_tokens = json.load(f)
else:
    other_tokens = {}

# Process each token in the list
for full_token_id, token_name in data.items():
    network, token_address = full_token_id.split('_')
    process_token(network, token_address, token_name, other_tokens)

# Display the updated token list
len(other_tokens)


HTTP error: 404 Client Error: Not Found for url: https://api.geckoterminal.com/api/v2/networks/eth/tokens/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48/pools?page=11
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.3%
Conflict detected for address eth_0xdac17f958d2ee523a2206206994597c13d831ec7: USDT vs USDT 0.01%
Conflict detected for address eth_0xdac17f958d2ee523a2206206994597c13d831ec7: USDT vs USDT 0.01%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.05%
Conflict detected for address eth_0xdac17f958d2ee523a2206206994597c13d831ec7: USDT vs USDT 0.05%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 1%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.03%
Conflict detected for address eth_0xdac17f958d2ee523a2206206994597c13d831ec7: USDT vs USDT 0.005%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ea

HTTP error: 429 Client Error: Too Many Requests for url: https://api.geckoterminal.com/api/v2/networks/eth/tokens/0xcd5fe23c85820f7b72d0926fc9b05b43e359b7ee/pools?page=8
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.05%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.01%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.3%
Conflict detected for address eth_0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: WETH vs WETH 0.01%
HTTP error: 429 Client Error: Too Many Requests for url: https://api.geckoterminal.com/api/v2/networks/eth/tokens/0xbf5495efe5db9ce00f80364c8b423567e58d2110/pools?page=1
HTTP error: 429 Client Error: Too Many Requests for url: https://api.geckoterminal.com/api/v2/networks/eth/tokens/0x6982508145454ce325ddbe47a25d4ec3d2311933/pools?page=1
HTTP error: 429 Client Error: Too Many Requests for url: https://api.geckoterminal.com/api/v2/networks/